
Feedback : shins777@gmail.com

이 Colab 은 Generative AI 호출시 동시성 제어를 위한 Async call 과 Pralllel call 을 구현한 예제입니다.
대량의 처리를 Concurrent 기반의 Latency를 줄이기 위해서 아래 예제를 참고하세요.

#라이브러리 설치
상세한 Python API 정보는 아래 URL 참고하세요.
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [32]:
!pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

#GCP 사용자 인증 / 환경설정

Colab 환경에서는 아래와 같이 인증이 필요합니다.

In [33]:
from google.colab import auth
auth.authenticate_user()

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_vertexai.llms import VertexAI

본인의 GCP 환경에 맞게 아래 설정을 구성하세요. 단, model_name은 "gemini-pro" 를 그대로 사용하세요.

In [35]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

# 모델 초기화 및 기본 실행
1. Langchain을 기반으로한 기본 Operation
2. Responsible AI
3. Stop word

구글은 아래와 같이 생성형 AI 처리시 Responsible AI 부분으로 Harmcategory 에 따른 LLM 응답 조절이 가능합니다.


Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold



In [36]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

*  https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro

아래 내용은 Langchain 기반에서의 gemini_pro 를 초기화하는 방법입니다.
업무에 따라서 Langchain 을 사용하셔도 좋고, 만일 latency 가 중요하다고 하면, Google 자체 API를 사용하는것도 Latency를 줄이는 방법이 될수 있습니다.

In [37]:
gemini_pro = VertexAI( model_name=model_name,
                  project=project,
                  location=location,
                  verbose=True,
                  streaming=True,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 10
                 )

# Asynch Call
비즈니스 로직상에서 Async call 이 필요한 요건에 활용.
Async 호출은 Latency를 줄이고, 아키텍처 유연성을 확보하기 위해서 중요한 호출 방식입니다.


#### 호출에 필요한 함수 정의

In [38]:
import time
import asyncio
import threading

# 현재 일반 사용자 Colab에서 2 core 밖에 없어서 Aysnc 처리시 아래 코드 필요.
# https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop-when-using-jupyter-no
import nest_asyncio
nest_asyncio.apply()

questions = ["한국의 수도는 어디인가요 ?","일본의 수도는 어디인가요 ?","미국의 수도는 어디인가요 ?","영국의 수도는 어디인가요 ?","프랑스의 수도는 어디인가요 ?"]

# Define a serial function
def synch_generate(llm, prompt):
  resp = llm.generate(prompts =[prompt])
  print(f"Thread:{threading.get_ident()} : {resp.generations[0][0].text}")

def synch_call(llm):
  result = [synch_generate(llm, questions[i]) for i in range(5)]

#------------------------------------------------

# Define an async function.
async def async_generate(llm, prompt):
  resp = await llm.agenerate(prompts =[prompt])
  print(f"Thread:{threading.get_ident()} : {resp.generations[0][0].text}")

async def generate_concurrently(llm):
  tasks = [async_generate(llm, questions[i] ) for i in range(5)]
  await asyncio.gather(*tasks)





#### Sync, Async 호출 비교

In [39]:

start = time.perf_counter()
synch_call(gemini_pro)
elapsed = time.perf_counter() - start
print(f"Serially executed in {elapsed:0.2f} seconds.\n")

start = time.perf_counter()
asyncio.run(generate_concurrently(gemini_pro))
elapsed = time.perf_counter() - start
print(f"Concurrently executed in {elapsed:0.2f} seconds.\n")

Thread:139504216526848 : 서울
Thread:139504216526848 : 도쿄
Thread:139504216526848 : 워싱턴 D.C.
Thread:139504216526848 : 런던
Thread:139504216526848 : 파리
Serially executed in 8.45 seconds.

Thread:139504216526848 : 서울
Thread:139504216526848 : 워싱턴 D.C.
Thread:139504216526848 : 도쿄
Thread:139504216526848 : 파리
Thread:139504216526848 : 런던
Concurrently executed in 2.96 seconds.



# Concurrent call
1. Parallel processing 일 필요.
2. Latency를 줄이기 위해서 동시에 처리해야 할 요건에 필요.

#### 호출에 사용되는 함수 정의

In [40]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

vertexai.init(project="ai-hangsik", location="us-central1")


def langchain_call(country):
  capital = gemini_pro.invoke(f"{country} 의 수도는 ?")
  print(f"Thread: {threading.get_ident()} : {capital}")
  return capital

def native_call(country):

  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    [f"{country} 의 수도는 ?"],
  )
  capital = responses.text

  print(f"Thread: {threading.get_ident()} : {capital}")
  return capital

#### 아래 코드는 순차적으로 호출할때의 latency를 측정하기 위한 목적입니다.

In [41]:
import time
import threading

#----[ Lanchain Call ]-----

t1 = time.time()

langchain_call("대한민국")
langchain_call("일본")
langchain_call("중국")

t2 = time.time()

print(t2-t1)

#----[ Native Call ]-----

t1 = time.time()

native_call("대한민국")
native_call("일본")
native_call("중국")

t2 = time.time()

print(t2-t1)



Thread: 139504216526848 : 서울
Thread: 139504216526848 : 도쿄
Thread: 139504216526848 : 베이징
4.988749027252197
Thread: 139504216526848 : 서울
Thread: 139504216526848 : 도쿄
Thread: 139504216526848 : 베이징
3.468798875808716


#### 아래 예제는 ThreadPoolExecutor 의 map function을 통한 호출을 테스트합니다.

In [42]:
from concurrent.futures import ThreadPoolExecutor

args_list ={"Korea", "Japan", "China"}

#----[ Lanchain Call ]-----

t1 = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(langchain_call, args_list)

for result in results:
  print(result)

t2 = time.time()

print(t2-t1)

#----[ Native Call ]-----

t1 = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(native_call, args_list)

for result in results:
  print(result)

t2 = time.time()

print(t2-t1)


Thread: 139498692261440 : 서울
Thread: 139498683868736 : 도쿄
Thread: 139498650297920 : 베이징
베이징
도쿄
서울
6.177240610122681
Thread: 139498641905216 : 도쿄
Thread: 139498650297920 : 베이징
Thread: 139498683868736 : 서울
베이징
도쿄
서울
1.6100623607635498


####아래 예제는 ThreadPoolExecutor 의 submit function을 통한 호출을 테스트합니다.

In [43]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

args_list ={"Korea", "Japan", "China"}

#----[ Lanchain Call ]-----

t1 = time.time()


with ThreadPoolExecutor(max_workers=10) as executor:

    futures = [executor.submit(langchain_call, arg) for arg in args_list]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)

t2 = time.time()

print(t2-t1)

#----[ Native Call ]-----

t1 = time.time()


with ThreadPoolExecutor(max_workers=10) as executor:

    futures = [executor.submit(native_call, arg) for arg in args_list]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)

t2 = time.time()

print(t2-t1)




Thread: 139498683868736 : 베이징
Thread: 139498641905216 : 서울
Thread: 139498650297920 : 도쿄
['베이징', '서울', '도쿄']
3.422769784927368
Thread: 139498650297920 : 서울
Thread: 139498641905216 : 도쿄
Thread: 139498683868736 : 베이징
['서울', '도쿄', '베이징']
2.7005832195281982
